For this case I will create a model to predict the consumption of a car based on its characteristics
-cylinders
-displacement
-horsepower
-weight
-acceleration
-model_year
-origin (It defines which country the car comes from) 
-mpg (this is the actual consume of the car)
Crea un modelo con él para que se pueda estimar el consumo de un vehículo a partir del resto de las variables.


Since we want to predict a numeric variable, I will build the model using Linear regression.

In [21]:
#I will start importing the libraries and data
import pandas as pd
import seaborn as sns
import numpy as np

data=pd.read_csv("auto.csv")
print(data)
y=data.mpg
data=data.drop(["mpg"],axis=1)

#I will take a look at the data
print(data.columns)
print(data.head())

     cylinders  displacement  horsepower  weight  acceleration  model_year  \
0            8         307.0       130.0  3504.0          12.0          70   
1            8         350.0       165.0  3693.0          11.5          70   
2            8         318.0       150.0  3436.0          11.0          70   
3            8         304.0       150.0  3433.0          12.0          70   
4            8         302.0       140.0  3449.0          10.5          70   
..         ...           ...         ...     ...           ...         ...   
387          4         140.0        86.0  2790.0          15.6          82   
388          4          97.0        52.0  2130.0          24.6          82   
389          4         135.0        84.0  2295.0          11.6          82   
390          4         120.0        79.0  2625.0          18.6          82   
391          4         119.0        82.0  2720.0          19.4          82   

     origin   mpg  
0         1  18.0  
1         1  15.0  
2  

# Dummy variables: 


With the "head" function I have been able to see that "cylinders" and the "origin" are two numerical variables that should be categorical, since they are just defining different classes.

For example, the variable "cylinders" determines if a car corresponds to the group of cars with 8 cylinders, with 4 cylinders... To have the information as a category is much more useful than making the model to calculate a "weight" to multiply for the number of cylinders the car has.

I will now proceed to examine how many unique numbers are in these two variables and then I will transform then in dummy variables.


In [22]:
print(data["cylinders"].unique())
print(data["origin"].unique())

[8 4 6 3 5]
[1 3 2]


In [23]:


#I will transform the variables in dummies with the function "OneHotEncoder"
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False) 

#Now I will introduce the variables I want to transform to the encoder that I have created. 
subdata=pd.DataFrame(ohe.fit_transform(data[['cylinders',"origin"]]))
print(ohe.categories_) #with the "categories_" I can check how the encoder has sorted the different categories 
    #[array([3, 4, 5, 6, 8], dtype=int64), array([1, 2, 3], dtype=int64)] #firs the cylinders with the following order 
    #3, 4, 5, 6, 8 and then the origins in this order 1,2,3
    
    
#Since the encoder has removes the name of the variables, I will now change the name of the columns.
subdata.columns=["3 cil", "4 cil", "5 cil", "6 cil", "8 cil","origen 1", "origen 2", "origen 3"]


#Now I will delete one column of each of the variables that I have transformed into dummies. 

#I do this because otherwise I the model will be obtaining the same information two times and thus it will only make the 
#prediction worst.


subdata=subdata.drop(["3 cil","origen 1"],axis=1) 
print(subdata.head())

#Now I will drop the two variables that I encoded that are in the "data" dataset,
#I do this because I will concatenate the "data" and the "subdata" dataframes. 
#If I didn't delete these two variables, then, again, I would be giving the same information two times to the model.

data=data.drop(["cylinders","origin"],axis=1)

[array([3, 4, 5, 6, 8], dtype=int64), array([1, 2, 3], dtype=int64)]
   4 cil  5 cil  6 cil  8 cil  origen 2  origen 3
0    0.0    0.0    0.0    1.0       0.0       0.0
1    0.0    0.0    0.0    1.0       0.0       0.0
2    0.0    0.0    0.0    1.0       0.0       0.0
3    0.0    0.0    0.0    1.0       0.0       0.0
4    0.0    0.0    0.0    1.0       0.0       0.0


# Data standardization

Before I proceed to concatenate the dataframes, I will standardize the information that is contained in the "data" dataframe. 

I'm doing this step because all the variables show different dimensions. 
For example, the variable "weight" has mean of 2977 and the variable "model_year" has a mean of 75. 
These two variables are not proportional and if the data was not scaled, then the model would give more importance 
to the variable with bigger numbers.

I'm applying this procedure BEFORE concatenating the datasets "data" and "subdata" because the data-science community it is not clear whether the dummy variables should be standardized or not.


In [24]:
from sklearn.preprocessing import StandardScaler
names=data.columns #I get the number of the columns, because the scaler will delete the names of the columns 
scal=StandardScaler()#I create the scaler
data=pd.DataFrame(scal.fit_transform(data)) #I apply the scaler
data.columns=names #I introduce the original names to the different columns

#I concatenate the two dataframes.
data=pd.concat([subdata, data],axis=1)

#Finally I check if everything went as expected. 
print(data.head())

   4 cil  5 cil  6 cil  8 cil  origen 2  origen 3  displacement  horsepower  \
0    0.0    0.0    0.0    1.0       0.0       0.0      1.077290    0.664133   
1    0.0    0.0    0.0    1.0       0.0       0.0      1.488732    1.574594   
2    0.0    0.0    0.0    1.0       0.0       0.0      1.182542    1.184397   
3    0.0    0.0    0.0    1.0       0.0       0.0      1.048584    1.184397   
4    0.0    0.0    0.0    1.0       0.0       0.0      1.029447    0.924265   

     weight  acceleration  model_year  
0  0.620540     -1.285258   -1.625315  
1  0.843334     -1.466724   -1.625315  
2  0.540382     -1.648189   -1.625315  
3  0.536845     -1.285258   -1.625315  
4  0.555706     -1.829655   -1.625315  


# Forward selection

Now I will select the variables that should be part of the lineal regression, using the "forward selection" method.

In [25]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

feature_order =  []
feature_error = []

features=data.columns
print(data.columns)


for i in range(len(features)):
    idx_try = [val for val in range(len(features)) if val not in feature_order]
    iter_error = []

    for i_try in idx_try:
        useRow = feature_order[:]
        useRow.append(i_try)

        use = data[data.columns[useRow]]

        model.fit(use, y)
        rmsError = np.linalg.norm((y - model.predict(use)), 2)/np.sqrt(len(y))
        iter_error.append(rmsError)
    
    pos_best = np.argmin(iter_error)
    feature_order.append(idx_try[pos_best])
    feature_error.append(iter_error[pos_best])

features_list=[]
for i in range(len(features)):
    print("In the step", i, "the following variable got introduced", 
        features[feature_order[i]], "with an error of", feature_error[i])
    features_list.append(str(features[feature_order[i]]))
print(features_list)



Index(['4 cil', '5 cil', '6 cil', '8 cil', 'origen 2', 'origen 3',
       'displacement', 'horsepower', 'weight', 'acceleration', 'model_year'],
      dtype='object')
In the step 0 the following variable got introduced weight with an error of 4.3216451262707
In the step 1 the following variable got introduced model_year with an error of 3.414013752452535
In the step 2 the following variable got introduced 6 cil with an error of 3.2557095011977686
In the step 3 the following variable got introduced horsepower with an error of 3.2249458938886733
In the step 4 the following variable got introduced origen 3 with an error of 3.194371955452919
In the step 5 the following variable got introduced origen 2 with an error of 3.169778591843068
In the step 6 the following variable got introduced displacement with an error of 3.1286746856188383
In the step 7 the following variable got introduced 4 cil with an error of 3.0860767239734743
In the step 8 the following variable got introduced 5 cil with 

Based on the error that show the different models, I can conclude that all the varaibles add usefull information to the model but the "acceleration" one. This last one makes the model worst and thus, it must be deleted.

In order to be sure about this conclusion I will also calculate the adjusted R^2 so I can compare the different models. 

I will calculate the adjusted R^2 of 4 models, those that seem to be more significant. 

In [26]:
#This model predict a good part of the variance but still there is a big part that can't be predicted.


x=data.loc[:,["weight","model_year","6 cil","horsepower","origen 3","origen 2"]]
model=LinearRegression()
model.fit(x,y)

print("R^2 modelo 1:",model.score(x,y))

R=model.score(x,y)
adj_r2 = (1 - (1 - R) * ((len(x) - 1) / 
          (len(x) - len(x.columns) - 1))) 
print("R^2 Ajustada:", adj_r2) 


print("")

#This second model is better than the last one, it shows a better R^2 and adjusted R^2
x=data.loc[:,["weight","model_year","6 cil","horsepower","origen 3","origen 2","displacement","4 cil","5 cil"]]
model=LinearRegression()
model.fit(x,y)

print("R^2 modelo 2:",model.score(x,y))
R=model.score(x,y)

adj_r2 = (1 - (1 - R) * ((len(x) - 1) / 
          (len(x) - len(x.columns) - 1))) 
print("R^2 Ajustada:", adj_r2) 

print("")


#This even better than the last one.
x=data.loc[:,["weight","model_year","6 cil","horsepower","origen 3","origen 2","displacement","4 cil","5 cil","8 cil"]]
model=LinearRegression()
model.fit(x,y)

print("R^2 modelo 3:",model.score(x,y))
R=model.score(x,y)

adj_r2 = (1 - (1 - R) * ((len(x) - 1) / 
          (len(x) - len(x.columns) - 1))) 
print("R^2 Ajustada:", adj_r2) 


print("")


#This forth model, even though it has a greater R^2 the adjusted R^2 gets worst. 
#I can conclude then that this model is worse than the previous one.


x=data.loc[:,["weight","model_year","6 cil","horsepower","origen 3","origen 2","displacement","4 cil","5 cil","8 cil","acceleration"]]
model=LinearRegression()
model.fit(x,y)

print("R^2 modelo 4:",model.score(x,y))
R=model.score(x,y)

adj_r2 = (1 - (1 - R) * ((len(x) - 1) / 
          (len(x) - len(x.columns) - 1))) 
print("R^2 Ajustada:",adj_r2)

print("")

R^2 modelo 1: 0.8346437869839413
R^2 Ajustada: 0.8320668070408339

R^2 modelo 2: 0.8444840614198668
R^2 Ajustada: 0.8408200733381359

R^2 modelo 3: 0.8468843168566745
R^2 Ajustada: 0.8428655325222041

R^2 modelo 4: 0.8469157299040813
R^2 Ajustada: 0.8424843431381468



In [27]:
#After considering this 4 models I can conclude that the best model is the following one.

x=data.loc[:,["weight","model_year","6 cil","horsepower","origen 3","origen 2","displacement","4 cil","5 cil","8 cil"]]
model=LinearRegression().fit(x,y)
print("R^2 modelo 3:",model.score(x,y))

R^2 modelo 3: 0.8468843168566745


# Overfitting

Now I will check if the model is overfitted.

In [28]:
from sklearn.model_selection import train_test_split

#I start dividing the dataset in training and testing data.
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=1)

#I'm now training the model with the training data
model=LinearRegression()
model.fit(x_train,y_train)


predic_train=model.predict(x_train)
predic_test=model.predict(x_test)

print("R^2 with training data: ",model.score(x_train,y_train))
print("R^2 with test data: ",model.score(x_test,y_test))
#The results don't show that the model is not Overfitted

R^2 with training data:  0.8445968083568386
R^2 with test data:  0.8479074244491961
